In [26]:
##This will find all the townships in the county, which you can use in the ratiostudy function
findtownships = function(county) {
    excluded23 = read.csv('tippecanoe/Combined_Reconciliation_2023.csv')
    excluded = excluded23[excluded23$County == county & excluded23$Reclassified == TRUE,]
    return (unique(excluded$Township))
}
findtownships('Tippecanoe')

[1] "Wea"        "Shelby"     "Lauramie"   "Wabash"     "Sheffield" 
 [6] "Perry"      "Fairfield"  "Union"      "Tippecanoe" "Randolph"  
[11] "Washington" "Wayne"      "Jackson"    NA

In [27]:
##This will find all the exclusions in the county, which you can use in the ratiostudy function
findexclusions = function(county) {
    excluded23 = read.csv('tippecanoe/Combined_Reconciliation_2023.csv')
    excluded = excluded23[excluded23$County == county & excluded23$Reclassified == TRUE,]
    return (unique(excluded$Explanation))
}
findexclusions('Tippecanoe')

[1] "OUTSIDE TIMEFRAME"                  "AG LAND"                           
 [3] "AV DOES NOT REFLECT MARKET VALUE"   "SOLD TO EXEMPT ORGANIZATION"       
 [5] "LOW SALE PRICE"                     "ADJACENT OWNER"                    
 [7] "POOR CONDITION"                     "UNIQUE SALE"                       
 [9] "INVESTMENT/RENTAL"                  "SOLD MORE THAN ONCE IN ONE YEAR"   
[11] "FORCED SALE"                        "PARTIAL INTEREST"                  
[13] "PRE-EXISTING RELATIONSHIP"          "SALE DOES NOT REFLECT MARKET VALUE"
[15] "DUPLICATE"                          "BUSINESS TRANSACTION"              
[17] "MULTI PARCEL SALE"                  "INSUFFICIENT TIME ON MARKET"       
[19] "NO REASON PROVIDED"

In [28]:
prd = function(df) {
mean_ratio = mean(df$SaleRatio, na.rm = TRUE)
weighted_mean_ratio = sum(df$CurrentAV, na.rm = TRUE) / sum(df$SalePrice, na.rm = TRUE)
prd = mean_ratio / weighted_mean_ratio
return (prd)
}

In [29]:
cod = function(df) {
    median_ratio <- median(df$SaleRatio, na.rm = TRUE)
    cod <- (sum(abs(df$SaleRatio - median_ratio), na.rm = TRUE) / 
           (nrow(df) * median_ratio)) * 100
    
    return(cod)
}


In [30]:
prb = function(df) {
    model <- lm(SaleRatio ~ SalePrice, data = df)
    
    # PRB is the slope of the regression model
    prb <- coef(model)[2]
    
    return(prb)
}

In [31]:
cov <- function(df) {
    mean_ratio <- mean(df$SaleRatio, na.rm = TRUE)
    sd_ratio <- sd(df$SaleRatio, na.rm = TRUE)
    cov_value <- (sd_ratio / mean_ratio) * 100
    return(cov_value)
}


In [32]:
#Input to ratiostudy as follows -> ratiostudy(County,Townships (as vector),Exclusions to add back (as vector),
#Ratio Study Type)
#Example -> ratiostudy('Tippecanoe',c('Tippecanoe','Union'),
#c('VACANT LAND','AV DOES NOT REFLECT MARKET VALUE','INVESTMENT/RENTAL','INSUFFICIENT TIME ON MARKET'),
#'Residential Improved')
combine = function(county,exclusionlist){
    excluded23 = read.csv('tippecanoe/Combined_Reconciliation_2023.csv')
    ratios23 = read.csv('tippecanoe/Combined_Study_2023.csv')
    ratios = ratios23[ratios23$County == county,]
    ratios$PostedExplanation = 'FROM RATIO STUDY'
    ratios$Explanation = 'FROM RATIO STUDY'
    excluded = excluded23[excluded23$County == county,]
    filtered_exc = excluded[excluded$Explanation %in% exclusionlist,]
    tojoin = filtered_exc[,1:12]
    combined = rbind(ratios,tojoin)
    return (combined)
}
ratiostudy = function(county,townships,exclusions,type) {
    countydf = combine(county,exclusions)
    df = countydf[countydf$Township %in% townships,]
    if(type == 'Commercial Vacant') {
        sales = df[df$PropertyClass == 300,]
        if(nrow(sales) <= 5){
            print(paste0('Only ',nrow(sales),' sales, not enough to study!'))
            return()
        }
        cat(paste0(type,' Study for ',df$County[1],' County, ',paste(townships, collapse = ", "),' Township(s):\n','Sales:',
                   nrow(sales),' Median Ratio:', round(median(sales$SaleRatio,na.rm=TRUE),2),
                   ' PRD:',round(prd(sales),2),' PRB:',round(prb(sales),2),' COD:',round(cod(sales),2),
                   ' COV:',round(cov(sales),2),'\n',
                   'Suggested Values:\n','Median Ratio: 0.9-1.1 PRD: 0.98-1.03 PRB: 0 COD: <20 COV: <15'))
    }
    if(type == 'Commercial Improved') {
        sales = df[df$PropertyClass > 300 & df$PropertyClass < 400,]
        if(nrow(sales) <= 5){
            print(paste0('Only ',nrow(sales),' sales, not enough to study!'))
            return()
        }
        cat(paste0(type,' Study for ',df$County[1],' County, ',paste(townships, collapse = ", "),' Township(s):\n','Sales:',
                   nrow(sales),' Median Ratio:', round(median(sales$SaleRatio,na.rm=TRUE),2),
                   ' PRD:',round(prd(sales),2),' PRB:',round(prb(sales),2),' COD:',round(cod(sales),2),
                   ' COV:',round(cov(sales),2),'\n',
                   'Suggested Values:\n','Median Ratio: 0.9-1.1 PRD: 0.98-1.03 PRB: 0 COD: <20 COV: <15'))
    }
    if(type == 'Industrial Vacant') {
        sales = df[df$PropertyClass == 400,]
        if(nrow(sales) <= 5){
            print(paste0('Only ',nrow(sales),' sales, not enough to study!'))
            return()
        }
        cat(paste0(type,' Study for ',df$County[1],' County, ',paste(townships, collapse = ", "),' Township(s):\n','Sales:',
                   nrow(sales),' Median Ratio:', round(median(sales$SaleRatio,na.rm=TRUE),2),
                   ' PRD:',round(prd(sales),2),' PRB:',round(prb(sales),2),' COD:',round(cod(sales),2),
                   ' COV:',round(cov(sales),2),'\n',
                   'Suggested Values:\n','Median Ratio: 0.9-1.1 PRD: 0.98-1.03 PRB: 0 COD: <20 COV: <15'))
    }
    if(type == 'Industrial Improved') {
        sales = df[df$PropertyClass > 400 & df$PropertyClass < 500,]
        if(nrow(sales) <= 5){
            print(paste0('Only ',nrow(sales),' sales, not enough to study!'))
            return()
        }
        cat(paste0(type,' Study for ',df$County[1],' County, ',paste(townships, collapse = ", "),' Township(s):\n','Sales:',
                   nrow(sales),' Median Ratio:', round(median(sales$SaleRatio,na.rm=TRUE),2),
                   ' PRD:',round(prd(sales),2),' PRB:',round(prb(sales),2),' COD:',round(cod(sales),2),
                   ' COV:',round(cov(sales),2),'\n',
                   'Suggested Values:\n','Median Ratio: 0.9-1.1 PRD: 0.98-1.03 PRB: 0 COD: <20 COV: <15'))
    }
    if(type == 'Residential Vacant') {
        sales = df[df$PropertyClass == 500,]
        if(nrow(sales) <= 5){
            print(paste0('Only ',nrow(sales),' sales, not enough to study!'))
            return()
        }
        cat(paste0(type,' Study for ',df$County[1],' County, ',paste(townships, collapse = ", "),' Township(s):\n','Sales:',
                   nrow(sales),' Median Ratio:', round(median(sales$SaleRatio,na.rm=TRUE),2),
                   ' PRD:',round(prd(sales),2),' PRB:',round(prb(sales),2),' COD:',round(cod(sales),2),
                   ' COV:',round(cov(sales),2),'\n',
                   'Suggested Values:\n','Median Ratio: 0.9-1.1 PRD: 0.98-1.03 PRB: 0 COD: <20 COV: <15'))
    }
    if(type == 'Residential Improved') {
        sales = df[df$PropertyClass > 500 & df$PropertyClass < 600,]
        if(nrow(sales) <= 5){
            print(paste0('Only ',nrow(sales),' sales, not enough to study!'))
            return
        }
        cat(paste0(type,' Study for ',df$County[1],' County, ',paste(townships, collapse = ", "),' Township(s):\n','Sales:',
                   nrow(sales),' Median Ratio:', round(median(sales$SaleRatio,na.rm=TRUE),2),
                   ' PRD:',round(prd(sales),2),' PRB:',round(prb(sales),2),' COD:',round(cod(sales),2),
                   ' COV:',round(cov(sales),2),'\n',
                   'Suggested Values:\n','Median Ratio: 0.9-1.1 PRD: 0.98-1.03 PRB: 0 COD: <20 COV: <15'))
    }
}
ratiostudy('Tippecanoe',c('Tippecanoe','Union'),
           c('VACANT LAND','AV DOES NOT REFLECT MARKET VALUE','INVESTMENT/RENTAL','INSUFFICIENT TIME ON MARKET'),
           'Residential Improved')

Residential Improved Study for Tippecanoe County, Tippecanoe, Union Township(s):
Sales:349 Median Ratio:0.89 PRD:1.05 PRB:0 COD:20.85 COV:62.71
Suggested Values:
Median Ratio: 0.9-1.1 PRD: 0.98-1.03 PRB: 0 COD: <20 COV: <15